## MNIST + Convolutional Neural Network

In [16]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.utils import to_categorical

In [17]:
img_rows = img_cols = 28
num_classes = 10

(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Reshape from (60000, 28,28) to (60000,28,28,1) : 60000 samples, 28x28 pixels, 1 channel (B/W)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
# Reshape from (10000, 28,28) to (10000,28,28,1) : 10000 samples, 28x28 pixels, 1 channel (B/W)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [16]:
print (x_train.shape)
print (y_train.shape)

(60000, 28, 28, 1)
(60000,)


In [17]:
# Normalize pixel values: [0-255] --> [0.0-1.0]
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels
# We could skip this step and use the 'sparse_categorical_crossentropy' loss function instead
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [18]:
print (y_train[:1])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [19]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               200832    
_________________________________________________________________
dens

In [20]:
batch_size = 128
epochs = 10

In [21]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 38s 631us/step - loss: 0.2275 - acc: 0.9322 - val_loss: 0.0788 - val_acc: 0.9755
Epoch 2/10
60000/60000 [==============================] - 37s 611us/step - loss: 0.0627 - acc: 0.9807 - val_loss: 0.0588 - val_acc: 0.9812
Epoch 3/10
60000/60000 [==============================] - 36s 607us/step - loss: 0.0455 - acc: 0.9862 - val_loss: 0.0446 - val_acc: 0.9856
Epoch 4/10
60000/60000 [==============================] - 37s 611us/step - loss: 0.0337 - acc: 0.9892 - val_loss: 0.0376 - val_acc: 0.9884
Epoch 5/10
60000/60000 [==============================] - 36s 607us/step - loss: 0.0275 - acc: 0.9913 - val_loss: 0.0312 - val_acc: 0.9893
Epoch 6/10
60000/60000 [==============================] - 36s 604us/step - loss: 0.0224 - acc: 0.9927 - val_loss: 0.0259 - val_acc: 0.9920
Epoch 7/10
60000/60000 [==============================] - 36s 608us/

In [22]:
score = model.evaluate(x_test, y_test)
print(score)

10000/10000 [==============================] - 3s 277us/step
[0.03256322299227722, 0.991]


In [24]:
model.save("cnn-mnist")

## Predict home-made digits

In [25]:
import numpy as np
import cv2

np.set_printoptions(precision=2, suppress=True)

def loadImage(filename):
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        #img = imread(filename, mode='L')
        img = img / 255
        # Reshape from (28,28) to (1,28,28,1) : 1 sample, 28x28 pixels, 1 channel (B/W)
        img = np.expand_dims(img, axis=0)
        img = np.expand_dims(img, axis=0)
        img = np.reshape(img, (1,img_cols,img_rows,1))
        return np.array(img)

In [26]:
from keras.models import load_model
model = load_model("cnn-mnist")
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               200832    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total para

In [31]:
img_rows = img_cols = 28
img = "9.png"
img = loadImage(img)
classes = model.predict(img)
predicted = np.argmax(classes)
result = ''
print(" *%s* %s %s" % (digit, classes, result))

 *9* [[0.   0.   0.11 0.   0.01 0.   0.   0.   0.   0.88]] 
